# Ensemble Learning

## Initial Imports

In [108]:
import warnings
warnings.filterwarnings('ignore')

In [109]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [110]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [111]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [112]:
# One Hot Encoding
df = df.apply(LabelEncoder().fit_transform)
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,1,0,2590,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,1,0,1889,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,1,0,2292,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,1,0,3010,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,1,0,1742,...,409,0,0,0,34824,11407,276,6250,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,329,19,3166,3,495,1,1,1,0,826,...,240,0,0,0,777,1000,113,520,0,0
68813,404,37,3414,3,2792,0,1,1,0,2773,...,379,0,0,0,17147,30243,135,27854,0,0
68814,135,27,1289,1,2073,1,1,1,0,1352,...,409,0,1,0,2327,6840,35,5433,0,0
68815,1397,1,8649,1,6019,2,1,1,0,862,...,399,34,0,0,59325,41120,1010,33176,0,0


## Split the Data into Training and Testing

In [113]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df['loan_status']

In [114]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,572.296903,12.982577,4126.615168,1.812779,3294.087856,0.669994,0.805542,0.0,1962.376855,0.217722,...,367.820219,57.069314,0.125972,0.0,27710.412107,24184.046704,295.617696,19966.952715,0.0,0.0
std,375.779218,9.508701,2367.621789,0.941313,1574.576387,0.719105,0.714932,0.0,971.922012,0.717338,...,67.053040,55.878931,0.336732,0.0,17546.728857,14497.653994,257.292302,13661.709989,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,289.000000,5.000000,2153.000000,1.000000,1930.000000,0.000000,0.000000,0.0,1255.000000,0.000000,...,347.000000,0.000000,0.000000,0.0,12014.000000,11764.000000,116.000000,7895.000000,0.0,0.0
50%,516.000000,11.000000,3828.000000,1.000000,3365.000000,1.000000,1.000000,0.0,1842.000000,0.000000,...,409.000000,48.000000,0.000000,0.0,26735.000000,23056.000000,221.000000,18429.000000,0.0,0.0
75%,848.000000,20.000000,5967.000000,3.000000,4665.000000,1.000000,1.000000,0.0,2532.000000,0.000000,...,409.000000,101.000000,0.000000,0.0,42861.000000,36149.000000,393.000000,31243.000000,0.0,0.0
max,1397.000000,51.000000,8809.000000,3.000000,6089.000000,2.000000,2.000000,0.0,5695.000000,17.000000,...,409.000000,155.000000,4.000000,0.0,59628.000000,51997.000000,1715.000000,46898.000000,0.0,0.0


In [115]:
X

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,0,2590,0,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,0,1889,0,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,0,2292,0,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,0,3010,0,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,0,1742,0,...,409,0,0,0,34824,11407,276,6250,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,329,19,3166,3,495,1,1,0,826,0,...,240,0,0,0,777,1000,113,520,0,0
68813,404,37,3414,3,2792,0,1,0,2773,0,...,379,0,0,0,17147,30243,135,27854,0,0
68814,135,27,1289,1,2073,1,1,0,1352,0,...,409,0,1,0,2327,6840,35,5433,0,0
68815,1397,1,8649,1,6019,2,1,0,862,0,...,399,34,0,0,59325,41120,1010,33176,0,0


In [116]:
# Check the balance of our target values
df['loan_status'].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [117]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [118]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
data_scaler

StandardScaler()

In [119]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train)

StandardScaler()

In [120]:
# Scale the training and testing data
X_train_scaled = data_scaler.transform(X_train)
X_test_scaled = data_scaler.transform(X_test)

In [121]:
X_train_scaled

array([[-0.64749254, -0.4186081 , -0.49897283, ...,  0.30231065,
         0.        ,  0.        ],
       [ 0.44670181, -0.62939056,  0.90953   , ..., -0.58181994,
         0.        ,  0.        ],
       [ 1.78909147, -0.84017301,  1.81496589, ..., -0.74285018,
         0.        ,  0.        ],
       ...,
       [ 0.83901052, -0.52399933,  1.30978344, ..., -0.44419101,
         0.        ,  0.        ],
       [-0.2498463 , -0.62939056,  0.07940607, ...,  0.59380317,
         0.        ,  0.        ],
       [-0.14843316, -1.15634669,  0.10690763, ..., -0.49538137,
         0.        ,  0.        ]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the following steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [122]:
# Resample the training data with the BalancedRandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf = rf.fit(X_train, y_train)

In [123]:
# Calculated the balanced accuracy score
predictions = rf.predict(X_test)

acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.6663161584297231

In [124]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,29,58
Actual 1,12,17106


In [125]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.71      0.33      1.00      0.45      0.58      0.31        87
          1       1.00      1.00      0.33      1.00      0.58      0.36     17118

avg / total       1.00      1.00      0.34      1.00      0.58      0.36     17205



In [126]:
# List the features sorted in descending order by feature importance
importances = rf.feature_importances_

importances_sorted = sorted(zip(rf.feature_importances_, X.columns), reverse=True)
print("Top 20 Important Features")
importances_sorted[:20]

Top 20 Important Features


[(0.09221642581945005, 'total_rec_prncp'),
 (0.06916241638945406, 'last_pymnt_amnt'),
 (0.0618766613650836, 'total_rec_int'),
 (0.05640233542880826, 'total_pymnt'),
 (0.05224191444301577, 'total_pymnt_inv'),
 (0.019668049914183097, 'dti'),
 (0.018487765914718957, 'installment'),
 (0.01757597588790243, 'out_prncp_inv'),
 (0.017474412796942576, 'mo_sin_old_rev_tl_op'),
 (0.01681202329811555, 'mo_sin_old_il_acct'),
 (0.016532635750652593, 'max_bal_bc'),
 (0.016226321311638158, 'out_prncp'),
 (0.015656115302157204, 'total_bal_ex_mort'),
 (0.015038910397363174, 'bc_util'),
 (0.014864887986300828, 'tot_cur_bal'),
 (0.014433875103802158, 'bc_open_to_buy'),
 (0.014238391796702715, 'mths_since_rcnt_il'),
 (0.0141228869481194, 'avg_cur_bal'),
 (0.013994765605835473, 'total_bal_il'),
 (0.01368162732395399, 'tot_hi_cred_lim')]

### Easy Ensemble Classifier

In [127]:
# Train the Classifier
# Easy ensemble classifier i.e. boosted tree - Gradient Boosted Classifier
from sklearn.ensemble import GradientBoostingClassifier
   
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=100,
                                    learning_rate=0.8,
                                    max_features=5,
                                    max_depth=3,
                                    random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)

In [131]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Balanced Accuracy Score : {acc_score}")

Balanced Accuracy Score : 0.6214349887797076


In [129]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,23,64
Actual 1,368,16750


In [130]:
# Print the imbalanced classification report
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.06      0.26      0.98      0.10      0.51      0.24        87
          1       1.00      0.98      0.26      0.99      0.51      0.28     17118

avg / total       0.99      0.97      0.27      0.98      0.51      0.28     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Balanced Random Forest Classifier at 0.6663161584297231

2. Which model had the best recall score?

    Balanced Random Forest Classifier at 1.00

3. Which model had the best geometric mean score?

    Balanced Random Forest Classifier at 0.58

4. What are the top three features?

    #1. 0.09221642581945005, 'total_rec_prncp'
    
    #2. 0.06916241638945406, 'last_pymnt_amnt'
    
    #3. 0.0618766613650836, 'total_rec_int'